# 简介

[Docker Python SDK](https://pypi.org/project/docker/)用于通过`Python`代码管理`Docker`容器、镜像、网络等`Docker`资源.


# 安装

In [1]:
! pip install docker

  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
Using cached docker-7.1.0-py3-none-any.whl (147 kB)



# 创建客户端

要使用`Docker Python SDK`，首先需要创建一个`Docker`客户端对象。

## 本地


默认情况下,`Docker`守护程序会侦听`Unix`套接字上的连接以接受来自本地客户端的请求,要连接到本地`Docker`的守护进程非常简单,使用`from_env`方法创建一个客户端对象。

In [ ]:
import docker

docker_client = docker.from_env()

## 远程

### 配置守护进程

要[访问远程主机上的`Docker`守护程序](https://docs.docker.com/engine/daemon/remote-access/),需要先配置守护程序以侦听`TCP`连接

1. 编辑`/lib/systemd/system/docker.service`文件,修改`ExecStart`行,加一个`-H tcp://0.0.0.0:2375`参数,如图:

![](https://github.com/cruldra/picx-images-hosting/raw/master/image.4g4fgdgr0v.png)

2. 重新加载`systemd`配置文件

```bash
sudo systemctl daemon-reload
```

3. 重启`Docker`服务

```bash
sudo systemctl restart docker
```

4. 验证

```bash
sudo netstat -lntp | grep dockerd
```

配置成功的话,会看到

![](https://github.com/cruldra/picx-images-hosting/raw/master/image.7ax3m5y232.png)

也可以在另外一台机器上使用`netcat`测试

```bash
nc -v host 2375
```

成功的话会看到:

![](https://github.com/cruldra/picx-images-hosting/raw/master/image.41xzpifqpr.png)

5. 防火墙

有些云服务商需要在安全组中开放`2375`端口,请根据实际情况配置。

### 创建客户端

In [5]:
import docker
docker_client = docker.DockerClient(base_url='tcp://8.130.104.39:2375')

# 打印远程服务器上正在运行的docker容器的id
print([container.id for container in docker_client.containers.list()])

['af0d6b44fa082f48e3d80553ba1f098e3ef455f93ae04aaa3e6582068af8413c', '22f1685e7996d54f722f83bfd02cf6d8fa40de6821f0762f090fa47aa7e3309c', 'f7d4fa3affd513d40167790ff5344b255437453d99698fd9cab77b442ec1923b']


### 使用`SSH`创建客户端

直接暴露`Docker`守护程序的`TCP`端口是不安全的,可以使用[`SSH`作为代理进行访问](https://blog.csdn.net/shirukai/article/details/125927428)

# 容器管理

## 在容器中运行一个命令

实现`docker run -it ubuntu bash -c "echo hello world"`的效果

In [7]:
command = 'bash -c "echo hello world"'

logs = docker_client.containers.run(
    "ubuntu",
    command,
    remove=True,  
    tty=True,
    stdin_open=True,
    # detach=True
)
print(logs.decode('utf-8').strip())

hello world


## 映射目录

### 守护进程在远程主机上

当`Docker`守护进程位于远程主机上时,需要先把位于本地主机上的文件传输到远程主机上,然后再映射到容器中.

可以创建一个上下文管理器来实现:

1. 接收一个本地目录路径
2. 和一个ssh配置
3. 在进入上下文的时候将本地目录打包传到ssh服务器上的指定目录下解压缩
4. 在退出上下文的时候删除远程服务器上的这个目录

In [1]:
! pip install paramiko

  Using cached bcrypt-4.2.0-cp39-abi3-win_amd64.whl.metadata (9.9 kB)
  Using cached cryptography-43.0.0-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached PyNaCl-1.5.0-cp36-abi3-win_amd64.whl.metadata (8.7 kB)
   ---------------------------------------- 0.0/226.2 kB ? eta -:--:--
   ----- ---------------------------------- 30.7/226.2 kB 1.3 MB/s eta 0:00:01
   -------------------------------- ------- 184.3/226.2 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 226.2/226.2 kB 2.3 MB/s eta 0:00:00
Using cached bcrypt-4.2.0-cp39-abi3-win_amd64.whl (151 kB)
Using cached cryptography-43.0.0-cp39-abi3-win_amd64.whl (3.1 MB)
Using cached PyNaCl-1.5.0-cp36-abi3-win_amd64.whl (212 kB)


In [2]:
from contextlib import contextmanager
import os
import tarfile
import tempfile

import paramiko


class SSHConfig:
    def __init__(self, hostname, username, password=None, key_filename=None, port=22):
        self.hostname = hostname
        self.username = username
        self.password = password
        self.key_filename = key_filename
        self.port = port


@contextmanager
def remote_directory(local_path, remote_path, ssh_config):
    # 创建SSH客户端
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        # 连接到远程服务器
        ssh.connect(
            ssh_config.hostname,
            port=ssh_config.port,
            username=ssh_config.username,
            password=ssh_config.password,
            key_filename=ssh_config.key_filename
        )

        # 创建一个临时目录来存储压缩文件
        with tempfile.TemporaryDirectory() as temp_dir:
            # 压缩本地目录
            tar_filename = os.path.join(temp_dir, "local_directory.tar.gz")
            with tarfile.open(tar_filename, "w:gz") as tar:
                tar.add(local_path, arcname=os.path.basename(local_path))

            # 创建SFTP客户端
            sftp = ssh.open_sftp()

            # 上传压缩文件
            remote_tar_path = os.path.join(remote_path, "local_directory.tar.gz")
            sftp.put(tar_filename, remote_tar_path)

            # 在远程服务器上解压文件
            _, stdout, _ = ssh.exec_command(f"mkdir -p {remote_path} && tar -xzf {remote_tar_path} -C {remote_path} && rm {remote_tar_path}")
            stdout.channel.recv_exit_status()  # 等待命令执行完成

        yield  # 允许在上下文中使用远程目录

    finally:
        # 退出上下文时，删除远程目录
        ssh.exec_command(f"rm -rf {remote_path}")
        
        # 关闭SSH连接
        ssh.close()

In [3]:
# 使用示例
if __name__ == "__main__":
    local_directory = r"D:\Workspace\aiworld\py_automation\digital_human_video\.data\videos\7385533550239419689"
    remote_directory_path = "/tmp/7385533550239419689"
    
    ssh_config = SSHConfig(
        hostname="8.130.104.39",
        username="root",
        password="dongjak@2015",  # 或使用 key_filename="/path/to/private/key"
    )

    with remote_directory(local_directory, remote_directory_path, ssh_config):
        print(f"本地目录 {local_directory} 已上传并解压到远程路径 {remote_directory_path}")
        # 在这里执行需要使用远程目录的操作
    
    print("上下文已退出，远程目录已被删除")

KeyboardInterrupt: 

In [9]:
import logging
import os

logger = logging.getLogger(__name__)
ffmpeg_image = "ffmpeg:1.0"
video_directory_path = r"D:\Workspace\aiworld\py_automation\digital_human_video\.data\videos\7385533550239419689"
print(video_directory_path)
video_directory = os.path.dirname(r"D:\Workspace\aiworld\py_automation\digital_human_video\.data\videos\7385533550239419689")
logs = (
        docker_client.containers.run(
            ffmpeg_image,
            command,
            volumes={video_directory: {"bind": "/tmp_app", "mode": "rw"}},
            remove=True,
            tty=True,
            stdin_open=True,
        )
        .decode("utf-8")
        .strip()
    )
logger.info(f"视频转音频: {logs}")

D:\Workspace\aiworld\py_automation\digital_human_video\.data\videos\7385533550239419689


APIError: 500 Server Error for http://8.130.104.39:2375/v1.43/containers/create: Internal Server Error ("invalid volume specification: 'D:\Workspace\aiworld\py_automation\digital_human_video\.data\videos:/tmp_app:rw'")